# 2-D  coverage and volumn

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
from scipy.stats import chi2

## 2-arm contextual bandit with $\epsilon$-greedy algorithm

In [2]:
# define reward function
def get_noise():
    return  np.random.normal(0,1,1)[0]

def get_unitv():
    # generate unit vector in sphere
    x = np.random.normal(0,1,2)
    return x/np.linalg.norm(x)


# y = 0.3*(x_1 + x_2) + noise
def ECB(n):
    # true parameter
    theta = np.array([0.3,0.3]).reshape(-1,1)
    
    # data matrix
    X = np.array([get_unitv() for i in range(10)])
    
    # contextual bandits - action set
    A = X.copy()
    
    y = X@theta + np.array([get_noise() for i in range(10)]).reshape(-1,1)
    
    y = y.reshape(-1)
    nums = np.ones(10)
    rewards = y.copy().reshape(-1)
    
    for i in range(n-10):
        arm_means = rewards/nums
        arm = np.argmax(arm_means)
        # defining decaying rate
        flag = np.random.uniform(0,1,1)[0]
        if flag > np.log(i+1)**2/(i+1) and i > 50:
            arm = np.random.randint(1,11) - 1
        
        # append new observation
        newx = A[arm,:]
        X = np.append(X, newx.reshape(1,-1), axis = 0 )
        newr = np.sum(newx*0.3)  + get_noise()
        y = np.append(y, newr)
        
    return X, y

In [3]:
def Volumn(V):
    # given a matrix V, compute the volumn correspoing to the ellipse of the format x^T V x \leq 1
    # universal constant are dropped
    s =  np.linalg.svd(V)[1]
    return 1/np.product(np.sqrt(s))

def scaled_estimators(n, lam, sampler):
    
    # consider confidence level 0.8, 0.85, 0.9
    cql = chi2.ppf([0.8, 0.85, 0.9], 2)
    true_theta = np.array([0.3, 0.3]).reshape(-1,1)
    CI_data = np.zeros((2,15))
    
    X,y = sampler(n)
    #######################################
    # compute OLS
    M = LinearRegression(fit_intercept = False).fit(X, y)
    coeff = M.coef_.copy().reshape(-1,1)
    Sn = X.T@X
    residual = y.reshape(-1,1) - X@coeff
    # compute residual and use it as an estimate of sigma^2
    sig_h = np.sqrt(np.mean(residual**2))
    
    c = 1/sig_h**2*(coeff.reshape(1,-1)- true_theta.reshape(1,-1))@Sn@(coeff - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = np.log(Volumn(Sn)*r)
        CI_data[:,0 + 5*i] = np.array([cover, v]).reshape(-1)
    
    #######################################
    # W-decorrelation
    W_lambdas = np.ones(n)*lam
    # Latest parameter estimate vector
    wols = M.coef_.copy()
    # Latest w_t vector
    w = np.zeros((2,1))
    # Latest matrix W_tX_t = w_1 x_1^T + ... + w_t x_t^T
    WX = np.zeros((2,2))
    # Latest vector of marginal variances reward_vars * (w_1**2 + ... + w_t**2)
    W = np.zeros_like(X)
    for t in range(n):
        # Update w_t = (1/(norm{x_t}^2+lambda_t)) (x_t - W_{t-1} X_{t-1} x_t)
        w = (np.eye(2) - WX)@X[t,:].reshape(-1,1) 
        w = w/(1.0 + W_lambdas[t])
        W[t, :] = w.reshape(-1)
        # Update beta_t = beta_{t-1} + w_t (y_t - <beta_OLS, x_t>)
        wols += w.reshape(-1) * (y[t] - np.sum(X[t,:]*coeff.reshape(-1)) ) ###
        # Update W_tX_t = W_{t-1}X_{t-1} + w_t x_t^T 
        WX = WX + w@X[t,:].reshape(1,-1)
        
    WW = np.linalg.inv(W.T@W)
    r = cq**1.5*sig_h**3
   
    # compute statistic
    c = 1/sig_h**2*(wols - true_theta.reshape(1,-1))@WW@(wols.reshape(-1,1) - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = np.log(Volumn(WW)*r)
        CI_data[:,1 + 5*i] = np.array([cover, v]).reshape(-1)
    
    
    
    ######################################
    #  ALEE
    Z = np.zeros_like(X)
    for t in range(n):
        if t == 0:
            S = np.log(n)*np.eye(2)
        elif t == 1:
            Xsub = X[:t,:].reshape(-1,2)
            S = np.log(n)*np.eye(2) + Xsub.T@Xsub
        else:
            Xsub = X[:t,:]
            S = np.log(n)*np.eye(2) + Xsub.T@Xsub
        
        S_invh = scipy.linalg.sqrtm(np.linalg.inv(S))
        
        zt = S_invh@X[t,:].reshape(-1,1)
        Z[t,:] = zt.reshape(-1)
    
    W = np.zeros_like(Z)
    for i in range(n):
        if i==0:
            z = Z[0,:].reshape(1,-1)
            vtt = np.eye(2)
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        elif i == 1:
            z = Z[:(i+1),:]
            vtt = vt
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        else:
            z = Z[:(i+1),:]
            vt = np.linalg.inv(np.eye(2) + z.T@z)
            z = Z[:i,:]
            vtt = np.linalg.inv(np.eye(2) + z.T@z)
        
        w = vt@Z[i,:].reshape(-1,1)
        
        w_factor = np.sqrt(1 + Z[i,:].reshape(1,-1)@vtt@Z[i,:].reshape(-1,1))
        w = w_factor.reshape(-1)[0]*w
        
        W[i,:] = w.reshape(-1)
    
    alee = np.linalg.inv(W.T@X)@W.T@y.reshape(-1,1)
    
    A = W.T@X
    OSn = A.T@A

    c = 1/sig_h**2*(alee.reshape(1,-1)- true_theta.reshape(1,-1))@OSn@(alee - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = np.log(Volumn(OSn)*r)
        CI_data[:,3 + 5*i] = np.array([cover, v]).reshape(-1)
    
    
    ######################################
    # self-normalized martingale
    
    rols = np.linalg.inv(X.T@X + np.eye(2))@X.T@y.reshape(-1,1)
    
    Vt = np.eye(2) + Sn
    r = sig_h*np.sqrt(np.log(100*np.linalg.det(Vt))) + 1
    alpha_list = [0.2, 0.15, 0.1]

    c = (rols.reshape(1,-1) - true_theta.reshape(1,-1))@Vt@(rols - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        alpha = alpha_list[i]
        r = sig_h*np.sqrt(np.log(np.linalg.det(Vt)/alpha**2)) + 1
        cover = c < r**2 - 0
        v = np.log(Volumn(Vt)*r**3)
        CI_data[:,4 + 5*i] = np.array([cover, v]).reshape(-1)
    
    return CI_data
    
#   replication function
def ACV_repli(N , n,  lam, sampler):
    E1 = np.zeros((N, 10))
    E2 = np.zeros((N, 10))
    E3 = np.zeros((N, 10))
    
    for i in range(N):
        output = scaled_estimators(n, lam, sampler)
        E1[i,:] =  output[:,:5].reshape(1,10)
        E2[i,:] =  output[:,5:10].reshape(1,10)
        E3[i,:] =  output[:,10:15].reshape(1,10)
        
    return (E1,E2,E3)

In [4]:
def compute_lam(N, n, sampler):
    
    record = np.zeros(N)
    for k in range(N):
        
        X,y = sampler(n)

        evalue, evector = np.linalg.eig(X.T@X)
        
        record[k] = np.min(evalue)
        
    return np.percentile(record, 10)

In [5]:
np.random.seed(517)
lam = compute_lam(1000, 1000, ECB)/np.log(1000)

In [6]:
np.random.seed(517)
E1,E2,E3 = ACV_repli(1000, 1000, lam, ECB)
np.save('E1', E1)
np.save('E2', E2)
np.save('E3', E3)

# Confidence level 0.8

## First row represents coverage ; second row represents average log (Volume)

In [7]:
df = pd.DataFrame(np.mean(E1, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.776,0.810,0.877,1.000
1,-4.404,-2.543,-3.114,-1.366


## Standard errors for empirical coverage and logarithm of volume

In [8]:
df = pd.DataFrame(np.std(E1, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.417,0.392,0.328,0.000
1,0.092,0.068,0.082,0.075


# Confidence level 0.85

In [9]:
df = pd.DataFrame(np.mean(E2, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.826,0.868,0.917,1.000
1,-4.157,-2.296,-2.867,-1.323


## Standard errors for empirical coverage and logarithm of volume

In [10]:
df = pd.DataFrame(np.std(E2, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.379,0.338,0.276,0.000
1,0.092,0.068,0.082,0.075


# Confidence level 0.9

In [11]:
df = pd.DataFrame(np.mean(E3, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.884,0.916,0.954,1.000
1,-3.867,-2.006,-2.577,-1.264


## Standard errors for empirical coverage and logarithm of volume

In [12]:
df = pd.DataFrame(np.std(E3, axis = 0).reshape(2,5)[:,[0,1,3,4]], columns = ['ols','W', 'alee', 'Concentration'])
round(df,3)

,ols,W,alee,Concentration
0,0.320,0.277,0.209,0.000
1,0.092,0.068,0.082,0.076
